In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
# !rm multinli_1.0.zip

In [3]:
from glob import glob

files = sorted(glob('multinli_1.0/*.jsonl'))
files

['multinli_1.0/multinli_1.0_dev_matched.jsonl',
 'multinli_1.0/multinli_1.0_dev_mismatched.jsonl',
 'multinli_1.0/multinli_1.0_train.jsonl']

In [4]:
import json

data = []
with open(files[0]) as fopen:
    for l in fopen:
        data.append(json.loads(l))

len(data)

10000

In [5]:
data[0]

{'annotator_labels': ['neutral',
  'entailment',
  'neutral',
  'neutral',
  'neutral'],
 'genre': 'slate',
 'gold_label': 'neutral',
 'pairID': '63735n',
 'promptID': '63735',
 'sentence1': 'The new rights are nice enough',
 'sentence1_binary_parse': '( ( The ( new rights ) ) ( are ( nice enough ) ) )',
 'sentence1_parse': '(ROOT (S (NP (DT The) (JJ new) (NNS rights)) (VP (VBP are) (ADJP (JJ nice) (RB enough)))))',
 'sentence2': 'Everyone really likes the newest benefits ',
 'sentence2_binary_parse': '( Everyone ( really ( likes ( the ( newest benefits ) ) ) ) )',
 'sentence2_parse': '(ROOT (S (NP (NN Everyone)) (VP (ADVP (RB really)) (VBZ likes) (NP (DT the) (JJS newest) (NNS benefits)))))'}

In [6]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('mesolitica/finetune-translation-t5-small-standard-bahasa-cased')
model = T5ForConditionalGeneration.from_pretrained('mesolitica/finetune-translation-t5-small-standard-bahasa-cased')

In [8]:
%%time

batch = [data[0]['sentence1'], data[0]['sentence2']]
t = ['terjemah Inggeris ke Melayu: ' + b for b in batch]
inputs = tokenizer(t, return_tensors="pt", padding = True)
translated_tokens = model.generate(**inputs, max_length=500)
decoded = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)

CPU times: user 482 ms, sys: 0 ns, total: 482 ms
Wall time: 62.1 ms


In [9]:
from tqdm import tqdm

with open('translated-mnli-dev_matched.jsonl', 'w') as fopenl:
    for d in tqdm(data):
        batch = [d['sentence1'], d['sentence2']]
        t = ['terjemah Inggeris ke Melayu: ' + b for b in batch]
        inputs = tokenizer(t, return_tensors="pt", padding = True)
        translated_tokens = model.generate(**inputs, max_length=500)
        decoded = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
        
        d['translate'] = decoded
        fopenl.write(f'{json.dumps(d)}\n')

100%|██████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [38:59<00:00,  4.27it/s]
